In [3]:
import equinox as eqx
import jax

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [8]:
print(type(jax.nn.sigmoid))

<class 'jaxlib.xla_extension.PjitFunction'>


In [9]:
class RNN(eqx.Module): 
    input_dim: int
    hidden_dim: int
    output_dim: int
    Wh: eqx.nn.Linear
    Wx: eqx.nn.Linear
    Wy: eqx.nn.Linear
    act: callable

    
    def __init__(self, input_dim=2, hidden_dim=2, output_dim=1):
        self.hidden_dim = hidden_dim
        self.Wh = eqx.nn.Linear(hidden_dim, hidden_dim)
        self.Wx = eqx.nn.Linear(input_dim, hidden_dim)
        self.Wy = eqx.nn.Linear(hidden_dim, output_dim)
        self.act = jax.nn.sigmoid

    def __call__(self, x): 
        batch_size = x.size(0)
        seq_length = x.size(1)

        hidden = torch.nn.Parameter(torch.zeros(batch_size, self.hidden_dim))
        outs = []

        for i in range(seq_length):
            hidden = self.act(self.Wh(hidden) + self.Wx(x[:,i,:]))
            out = self.Wy(hidden)
            outs.append(out)
        
        # output shape = (batch_size, sequence length, output dimension)
        return torch.stack(outs).permute(1,0,2)
